In [1]:
import cv2

def show_camera_feed():
    # Replace with your camera's stream URL
    camera_stream_url = '192.168.100.65'

    # Create a VideoCapture object
    cap = cv2.VideoCapture(camera_stream_url)

    # Check if the camera opened successfully
    if not cap.isOpened():
        print("Error: Could not open video stream.")
        return

    # Read and display frames from the camera
    while True:
        ret, frame = cap.read()
        if ret:
            cv2.imshow('Camera Feed', frame)
            # Press 'q' to exit the window
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            print("Error: Could not read frame.")
            break

    # Release the VideoCapture object and close the window
    cap.release()
    cv2.destroyAllWindows()

show_camera_feed()


Error: Could not open video stream.


In [3]:
import requests

r = requests.get('http://192.168.100.65')

In [2]:
dic = {'hi': 'hello'}

In [ ]:
dic.

In [5]:
import requests


res = requests.get('https://face-id.up.railway.app/test')

res

<Response [503]>

In [6]:
print(type(faceobjs))

<class 'list'>


In [18]:
faceobjs[0].iloc[0, 0]



'../dataset/Tarmica/image_1c13626ce0d94befac4135b660308117.jpg'

In [ ]:
import json
import boto3
from deepface import DeepFace

# Initialize the S3 client
s3_client = boto3.client('s3')

def lambda_handler(event, context):
    try:
        # Parse the incoming request to get the image data
        image_data = event['body']
        
        # Detect the face in the image using DeepFace
        face_detected = DeepFace.extract_faces(image_data, enforce_detection=False)
        
        if face_detected[0]['confidence']>0.7:
            # If a face is detected, search for the face in the S3 bucket
            db_path = "s3://chumbucketzw/images/"
            recognition_result = DeepFace.find(img_path=image_data, db_path=db_path, enforce_detection=False)
            faces_df = recognition_result[0]
            tup = recognition_result[0].shape
            
            if tup[0]>1:
                identity = faces_df.iloc[0,0]  # Extract the person's name
                person_name = identity.split('/')[-2]
                return {
                    'statusCode': 200,
                    'body': json.dumps(f'Person identified: {person_name}')
                }
            else:
                # Face not recognized
                return {
                    'statusCode': 200,
                    'body': json.dumps('Unknown human detected')
                }
        else:
            # No face detected
            return {
                'statusCode': 200,
                'body': json.dumps('No face is in the image')
            }
    except Exception as e:
        return {
            'statusCode': 500,
            'body': json.dumps(f'Error processing the image: {str(e)}')
        }
